In [33]:
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as pt
import re
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize


In [50]:
df=pd.read_csv('C:/Users/jyothi/Downloads/traintweet.csv')


In [35]:
positive = df[df['label']==0]
negative = df[df['label']==1]

In [36]:
def words(tweets):
    l = []
    for tweet in tweets:
        l.extend(str(tweet).split())
    return l

In [37]:
def get_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    features = wordlist.keys()
    return features
w_features = get_features(words(df['tweet']))

In [38]:
def e_features(document):
    document_words = set(document)
    features = {}
    for word in w_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [39]:
wordfile = pd.read_csv('/Users/jyothi/Downloads/words.csv')
wordfile=wordfile[::-1]
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    
    return input_txt

In [40]:
df['cleaned_tweets'] = np.vectorize(remove_pattern)(df['tweet'], "@[\\w]*")
df['cleaned_tweets'] = df['cleaned_tweets'].str.replace("[^a-zA-Z#]", " ")
df['cleaned_tweets'] = df['cleaned_tweets'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
tokenized_tweet = df['cleaned_tweets'].apply(lambda x: x.split())
tokenized_tweet.head()
stemmer = PorterStemmer()
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])

In [47]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

wordfile['sentiment']=le.fit_transform(wordfile['sentiment'])
wordfile

,Words,sentiment
7075,interesting,1
7074,yum,1
7073,glad,1
7072,love,1
7071,surprised,1
7070,pleasantly,1
7069,comeback,1
7068,rocks,1
7067,zany,1
7066,zealous,1


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
# list of text documents

# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(wordfile['Words'])
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector = vectorizer.transform(wordfile['Words'])
# summarize encoded vector
print(vector.shape)
print(vector.toarray()) 




{'interesting': 3529, 'yum': 6851, 'glad': 2681, 'love': 3888, 'surprised': 5990, 'pleasantly': 4651, 'comeback': 915, 'rocks': 5215, 'zany': 6853, 'zealous': 6859, 'wonderful': 6777, 'witty': 6764, 'vivacious': 6660, 'victorious': 6625, 'thoughtful': 6106, 'thankful': 6096, 'successful': 5924, 'splendid': 5695, 'smiling': 5583, 'silly': 5486, 'relieved': 5050, 'proud': 4835, 'pleasant': 4650, 'perfect': 4570, 'obedient': 4306, 'nicely': 4264, 'nice': 4263, 'lucky': 3908, 'lovely': 3893, 'lively': 3849, 'kind': 3713, 'joyous': 3684, 'jolly': 3678, 'hilarious': 2976, 'responsive': 5120, 'agile': 161, 'high': 2975, 'helpful': 2956, 'healthy': 2923, 'happy': 2851, 'pretty': 4755, 'good': 2722, 'glorious': 2705, 'gentle': 2663, 'funny': 2607, 'fun': 2603, 'friendly': 2562, 'fine': 2411, 'fantastic': 2299, 'faithful': 2261, 'fair': 2257, 'like': 3832, 'excellent': 2155, 'exuberant': 2230, 'excited': 2163, 'enthusiastic': 2067, 'energetic': 2019, 'encouraging': 2005, 'enchanting': 2001, 'ela

In [43]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
fm=vector
tv=wordfile['sentiment']
print(fm.shape)

print(tv.shape)

(7076, 6865)
(7076,)


In [44]:

model.fit(X=fm,y=tv)

c:\users\jyothi\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
result=[]
for i in df['cleaned_tweets']:
    n=[i]
    
    vector1 = vectorizer.transform(n)
    print(vector1.toarray())
    fm=vector1
    a=model.predict(fm)
    if 1 in a:
        print('THIS IS POSITIVE')
        result.append('YES')
    else:
        print('OOPS NOT POSITIVE')
        result.append('NO')

[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT P

[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS PO

[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITI

OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0.

[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITI

...

[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[

[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT P

[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT P

[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT PO

[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
THIS IS POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NOT POSITIVE
[[0. 0. 0. ... 0. 0. 0.]]
OOPS NO

In [51]:
df['POSITIVE?']=result
df

,id,label,tweet,POSITIVE?
0,1,0,@user when a father is dysfunctional and is s...,NO
1,2,0,@user @user thanks for #lyft credit i can't us...,NO
2,3,0,bihday your majesty,NO
3,4,0,#model i love u take with u all the time in ...,NO
4,5,0,factsguide: society now #motivation,NO
5,6,0,[2/2] huge fan fare and big talking before the...,NO
6,7,0,@user camping tomorrow @user @user @user @use...,NO
7,8,0,the next school year is the year for exams.ð...,NO
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,YES
9,10,0,@user @user welcome here ! i'm it's so #gr...,NO
